# Test monthly average

First make up some data and calculate the averages, then try using my function, and see you get the same answer.

In [9]:
#import my functions
import sys
sys.path.append(sys.path[0]+'/../..')


import helpers.fileHandler as fh
import utils.sstIndex as sst
import utils.pslIndex as psl
import utils._indexDefinitions as _index
from utils.timePeriod import averageForTimePeriod

In [2]:
import xarray
import numpy
import cftime
import pandas

I am going to make up some data, and put it in a pandas array first

In [3]:
#first make up some time period
dateRange=xarray.cftime_range(
    start=cftime.DatetimeNoLeap(1995,1,1),
    end=cftime.DatetimeNoLeap(1999,12,1),
    freq='M')

In [4]:
#put it into a pd Dataframe
testPd=pandas.DataFrame(index=dateRange)

testPd.index.name='time'

In [5]:
#add some rando data
for keys in _index.monthsOfInterest:
    testPd[keys]=5*numpy.random.rand(len(dateRange),1)

I am now going to calculate the averages in a really easy to understand way

In [6]:
#years of interest
expectedAv=pandas.DataFrame(index=[1995,1996,1997,1998])
expectedAv.index.name='year'

#for every index defined in the index file
for keys in ['nino12', 'nino3', 'nino34', 'nino4', 'westIO', 'eastIO', 'dmi', 'sam']:
    answer = list()
    
    #grab the time period defined in the index file
    #this line is a bit lazy, i really should define these indeces some other way incase of systematic errors
    monthPeriod=_index.monthsOfInterest[keys].copy()

    #select the data for this index
    indexArray = testPd[keys]
    
    temp = 0
    #for every month in the data
    for monthIter in numpy.arange(0,len(indexArray),1):
        # a month in the time period of interest:
        if ((monthIter+1)>=monthPeriod[0] and (monthIter+1)<=monthPeriod[1]):
            temp += indexArray.values[monthIter]
        else:
        # not a month in the time period of interest
            if temp!=0:
                answer.append(temp/(monthPeriod[1]-monthPeriod[0]+1))
                temp = 0
                monthPeriod[0]+=12
                monthPeriod[1]+=12
    #save output for this index and move to the next one
    expectedAv[keys]=answer


expectedAv.round(5)

,nino12,nino3,nino34,nino4,westIO,eastIO,dmi,sam
year,,,,,,,,
1995,2.57709,2.67896,2.50921,3.04636,2.66157,2.04437,2.53634,2.25569
1996,3.27948,2.54613,1.86305,2.29522,1.77373,2.50348,2.19254,2.50402
1997,3.49437,3.14645,2.91502,2.15064,2.26010,1.95727,2.54142,3.18623
1998,1.87163,2.75486,2.13854,2.58817,2.61851,2.47701,2.13698,2.62151


Calculate it using the function

In [7]:
#first cast it into an xarray
testXr = xarray.Dataset(testPd)

# do the calculation
calculatedAvs=averageForTimePeriod(testXr)
calculatedAvs.round(5)

,nino12,nino3,nino34,nino4,westIO,eastIO,dmi,sam
year,,,,,,,,
1995,2.57709,2.67896,2.50921,3.04636,2.66157,2.04437,2.53634,2.25569
1996,3.27948,2.54613,1.86305,2.29522,1.77373,2.50348,2.19254,2.50402
1997,3.49437,3.14645,2.91502,2.15064,2.26010,1.95727,2.54142,3.18623
1998,1.87163,2.75486,2.13854,2.58817,2.61851,2.47701,2.13698,2.62151


In [8]:
calculatedAvs==expectedAv

,nino12,nino3,nino34,nino4,westIO,eastIO,dmi,sam
year,,,,,,,,
1995,False,True,False,True,True,True,False,True
1996,True,False,True,True,True,True,False,True
1997,False,False,True,False,True,True,False,True
1998,False,False,False,True,True,True,False,True


These values are the same, but somewhere in there the last digit (of a float64) ended up different. Not overly significant.